In [3]:
# Library
!pip install SpeechRecognition pydub numpy #ffmpeg
!pip install openai-whisper spacy
!python -m spacy download es_core_news_md
!python -m spacy download en_core_web_md
!pip install mysql-connector-python gtts pyttsx3
!pip uninstall ffmpeg imageio-ffmpeg -y
!pip install ffmpeg-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 13.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Bot de reserva

## Captura audio reserva

In [14]:
import IPython
import uuid
from google.colab import output
from base64 import b64decode
import os

# Crear la carpeta "Original_recording" si no existe
os.makedirs("./Bot_NLP/original_recording", exist_ok=True)

AUDIO_HTML = """
<script>
var my_audio = "";
var recorder, gumStream;

function startRecording() {
    navigator.mediaDevices.getUserMedia({ audio: true }).then(function(stream) {
        gumStream = stream;
        recorder = new MediaRecorder(stream);
        recorder.ondataavailable = function(event) {
            var reader = new FileReader();
            reader.readAsDataURL(event.data);
            reader.onloadend = function() {
                var base64data = reader.result.split(',')[1];
                google.colab.kernel.invokeFunction('notebook.save_audio', [base64data], {});
            }
        };
        recorder.start();
    });
}

function stopRecording() {
    recorder.stop();
    gumStream.getAudioTracks()[0].stop();
}
</script>

<button onclick="startRecording()"> Iniciar Grabación</button>
<button onclick="stopRecording()"> Detener Grabación</button>
"""

IPython.display.display(IPython.display.HTML(AUDIO_HTML))

# Función para guardar el audio en un archivo
def save_audio(base64data):
    audio_filename = f"./Bot_NLP/original_recording/audio_{uuid.uuid4().hex}.wav"
    with open(audio_filename, "wb") as audio_file:
        audio_file.write(b64decode(base64data))
    print(f"🎙 Audio guardado en: {audio_filename}")
    return audio_filename

output.register_callback('notebook.save_audio', save_audio)

## Conversión de audio al formato .wav

In [3]:
import os
import glob
from pydub import AudioSegment

# Rutas de las carpetas
original_folder = "./Bot_NLP/original_recording"
converted_folder = "./Bot_NLP/converted_recording"

# Crear la carpeta si no existe
os.makedirs(converted_folder, exist_ok=True)

# Buscar archivos de audio en la carpeta original
audio_files = glob.glob(f"{original_folder}/audio_*.wav")
print(f"Archivos de audio encontrados: {audio_files}")

# Función para convertir el audio a PCM WAV
def convert_to_wav_pcm(audio_path):
    filename = os.path.basename(audio_path).replace(".wav", "_converted.wav")
    output_path = os.path.join(converted_folder, filename)

    try:
        # Cargar el archivo de audio y convertirlo
        audio = AudioSegment.from_file(audio_path)
        audio = audio.set_channels(1).set_frame_rate(16000)
        audio.export(output_path, format="wav")
        print(f"Archivo convertido y guardado en: {output_path}")
        return output_path
    except Exception as e:
        print(f"Error al convertir el audio {audio_path}: {e}")
        return None

# Convertir los archivos de audio y guardarlos en la nueva carpeta
converted_audio_paths = [convert_to_wav_pcm(audio) for audio in audio_files]
converted_audio_paths = [path for path in converted_audio_paths if path is not None]

📂 Archivos de audio encontrados: ['./Bot_NLP/original_recording/audio_1b621ff126b341fdb8cedb63a53bdd68.wav']
✅ Archivo convertido y guardado en: ./Bot_NLP/converted_recording/audio_1b621ff126b341fdb8cedb63a53bdd68_converted.wav


## Funciones para realizar la creación de JSON de la reserva

In [4]:
import spacy
import json
import pandas as pd
import re
import unicodedata

# Cargar modelos de NLP en inglés y español
nlp_en = spacy.load("en_core_web_sm")
nlp_es = spacy.load("es_core_news_md")

# Filtro de aeropuertos válidos
TIPOS_AEROPUERTO_VALIDOS = {"large_airport", "medium_airport", "small_airport"}

# Lista de aerolíneas conocidas
AEROLINEAS_CONOCIDAS = {
    "iberia", "avianca", "latam", "lufthansa", "ryanair", "air france", "american airlines", "delta",
    "united airlines", "british airways", "emirates", "qatar airways", "klm", "turkish airlines", "copa airlines"
}

# Quita los acentos a las ciudades para poder usar el archivo que contiene códigos IATA
def quitar_acentos(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn').lower().strip()

def cargar_ciudades(csv_path):
    df = pd.read_csv(csv_path, dtype=str)
    df = df[df["type"].isin(TIPOS_AEROPUERTO_VALIDOS)]
    df = df.dropna(subset=["municipality"])
    df["municipality"] = df["municipality"].apply(quitar_acentos)
    return set(df["municipality"])

# Ruta al CSV de aeropuertos
csv_path = "./airport-codes.csv"

# Cargar la lista de ciudades
ciudades_base = cargar_ciudades(csv_path)

def extraer_ciudades_con_posiciones(texto):
    texto_lower = quitar_acentos(texto)
    ciudades_detectadas = []
    # Buscar ciudades en el texto y guardar su posición
    for ciudad in ciudades_base:
        match = re.search(rf"\b{ciudad}\b", texto_lower)
        if match:
            ciudades_detectadas.append((ciudad.title(), match.start()))
    # Ordenar por posición en el texto para preservar el orden
    ciudades_detectadas.sort(key=lambda x: x[1])
    # Extraer nombres de aerolíneas detectadas para excluirlas de ciudades
    aerolinea_detectada = extraer_aerolinea(texto)
    # Filtrar ciudades para eliminar la aerolínea detectada
    ciudades_filtradas = [c[0] for c in ciudades_detectadas if c[0] != aerolinea_detectada]
    return ciudades_filtradas

def extraer_fecha(texto):
    patron_fecha = r"\b(\d{1,2}) de (enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre) (?:del|de) (\d{4})\b"
    match = re.search(patron_fecha, texto.lower())
    return f"{match.group(1)} de {match.group(2).capitalize()} de {match.group(3)}" if match else ""

def extraer_tickets(texto):
    patron_tickets = r"(\b(?:un|uno|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|\d+)\b)\s*(tickets|boletos|pasajes|flights|billetes|vuelos|ticket|boleto|pasaje|flight|billete|vuelo)"
    match = re.search(patron_tickets, texto.lower())

    if match:
        cantidad = match.group(1).lower()
        # Diccionario para convertir palabras en números
        num_palabras = {
            "un": 1, "uno": 1, "dos": 2, "tres": 3, "cuatro": 4, "cinco": 5,
            "seis": 6, "siete": 7, "ocho": 8, "nueve": 9, "diez": 10
        }
        if cantidad in num_palabras:
            return num_palabras[cantidad]

        if cantidad.isdigit():
            return int(cantidad)
    return 1

def extraer_aerolinea(texto):
    texto_lower = quitar_acentos(texto)

    for aerolinea in AEROLINEAS_CONOCIDAS:
        if re.search(rf"\b{aerolinea}\b", texto_lower):
            return aerolinea.title()
    return "No Especificado"

def ordenar_origen_destino(texto, ciudades_detectadas):
    texto_lower = quitar_acentos(texto)

    if len(ciudades_detectadas) >= 2:
        # Si se menciona "desde X a Y", usamos esas ciudades como referencia
        match = re.search(r"desde\s+(\w+)\s+a\s+(\w+)", texto_lower)
        if match:
            origen, destino = match.groups()
            origen = origen.title()
            destino = destino.title()
            if origen in ciudades_detectadas and destino in ciudades_detectadas:
                return origen, destino
        return ciudades_detectadas[:2]
    return ["Sin Aeropuerto", "Sin Aeropuerto"]

def extraer_informacion_a_json(texto):
    info_reserva = {
        "source": "No Especificado",
        "destination": "No Especificado",
        "tickets": 1,
        "date": "",
        "airline": "No Especificado"
    }

    # Extraer fecha, boletos y aerolínea
    info_reserva["date"] = extraer_fecha(texto)
    info_reserva["tickets"] = extraer_tickets(texto)
    info_reserva["airline"] = extraer_aerolinea(texto)

    # Extraer ciudades con posiciones en el texto
    ciudades_detectadas = extraer_ciudades_con_posiciones(texto)

    # Ordenar correctamente origen y destino
    info_reserva["source"], info_reserva["destination"] = ordenar_origen_destino(texto, ciudades_detectadas)

    return json.dumps(info_reserva, indent=4, ensure_ascii=False)


## Realiza el proceso de transcripción

In [9]:
import whisper
import os

# Definir rutas de las carpetas
ruta_audios = "Bot_NLP/converted_recording"
ruta_textos = "Bot_NLP/text_recording"

# Crear la carpeta si no existe
os.makedirs(ruta_textos, exist_ok=True)

# Cargar modelo de Whisper
model = whisper.load_model("medium")

def transcribir_audio(audio_path, output_path):
    print(f"\nTranscribiendo: {audio_path} ...")
    result = model.transcribe(audio_path)
    texto_transcrito = result["text"]
    print("Transcripción:")
    print(texto_transcrito)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(texto_transcrito)
    print(f"Transcripción guardada en: {output_path}")

# Recorrer los archivos de la carpeta de audios
for archivo in os.listdir(ruta_audios):
    if archivo.endswith((".wav")):
        audio_path = os.path.join(ruta_audios, archivo)
        texto_path = os.path.join(ruta_textos, os.path.splitext(archivo)[0] + ".txt")
        transcribir_audio(audio_path, texto_path)

print("\nProceso de transcripción completado.")

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)



🎙️ Transcribiendo: Bot_NLP/converted_recording/audio_1b621ff126b341fdb8cedb63a53bdd68_converted.wav ...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


📝 Transcripción:
 Buenas noches, por favor necesito un vuelo para Quito Guayaquil para la fecha del 25 de enero del 2025.
✅ Transcripción guardada en: Bot_NLP/text_recording/audio_1b621ff126b341fdb8cedb63a53bdd68_converted.txt

🎉 Proceso de transcripción completado.


## Creación del archivo JSON de la reserva

In [26]:
import os
import json

# Rutas de carpetas
transcription_folder = "Bot_NLP/text_recording/"
json_output_folder = "Bot_NLP/json_recording/"

# Crear la carpeta si no existe
os.makedirs(json_output_folder, exist_ok=True)

# Leer las transcripciones
for filename in os.listdir(transcription_folder):
    if filename.endswith(".txt"):
        file_path = os.path.join(transcription_folder, filename)
        # Leer contenido del archivo
        with open(file_path, "r", encoding="utf-8") as file:
            texto_transcrito = file.read().strip()
        # Generar JSON desde el texto transcrito
        json_reserva = extraer_informacion_a_json(texto_transcrito)
        # Definir el nombre del archivo JSON
        json_filename = os.path.splitext(filename)[0] + ".json"
        json_path = os.path.join(json_output_folder, json_filename)
        # Guardar el JSON en la carpeta de salida
        with open(json_path, "w", encoding="utf-8") as json_file:
            json.dump(json.loads(json_reserva), json_file, indent=4, ensure_ascii=False)

        print("JSON de Reserva:", extraer_informacion_a_json(texto_transcrito))
        print(f"\n JSON guardado: {json_path}")

🔹 JSON de Reserva: {
    "source": "Quito",
    "destination": "Guayaquil",
    "tickets": 1,
    "date": "25 de Enero de 2025",
    "airline": "No Especificado"
}

✅ JSON guardado: Bot_NLP/json_recording/audio_1b621ff126b341fdb8cedb63a53bdd68_converted.json


## Almacena los resultados del JSON de reserva en la base de datos

In [27]:
import mysql.connector
import json

# Configurar los datos de conexión
conexion = mysql.connector.connect(
    # Endpoint de SingleStore Cloud
    host="svc-15fdff82-6c53-40e2-a4de-19c59e39b17a-dml.aws-oregon-4.svc.singlestore.com",
    user="admin",
    password="rdme6tYm4qYA35cZzaVYLZHDn6rRCVfP",
    database="Bot_NLP",
    port=3306
)

cursor = conexion.cursor()

# Ruta de la carpeta donde se guardan los archivos JSON
json_folder = "Bot_NLP/json_recording/"

if not os.path.exists(json_folder):
    print("La carpeta de JSON no existe.")
    exit()

for filename in os.listdir(json_folder):
    if filename.endswith(".json"):
        json_path = os.path.join(json_folder, filename)
        # Leer contenido del archivo JSON
        with open(json_path, "r", encoding="utf-8") as json_file:
            data = json.load(json_file)
        # Extraer datos del JSON
        source = data.get("source", "Sin Aeropuerto")
        destination = data.get("destination", "Sin Aeropuerto")
        tickets = data.get("tickets", 1)
        date = data.get("date", "")
        airline = data.get("airline", "No Especificado")

        # Consulta SQL para insertar en la tabla
        sql = """
        INSERT INTO reservas_vuelos (source, destination, tickets, date, airline)
        VALUES (%s, %s, %s, %s, %s)
        """

        # Ejecutar la consulta con los datos
        cursor.execute(sql, (source, destination, tickets, date, airline))
        print(f"Insertado: {filename} -> {source} a {destination}, {tickets} tickets con {airline}")

# Confirmar los cambios en la base de datos
conexion.commit()

# Cerrar conexión
cursor.close()
conexion.close()

print("\n Todos los datos se han insertado en SingleStore.")

InterfaceError: 2003: Can't connect to MySQL server on 'svc-15fdff82-6c53-40e2-a4de-19c59e39b17a-dml.aws-oregon-4.svc.singlestore.com:3306' (Errno -2: Name or service not known)

## Genera la confirmación de la reserva

In [28]:
import os
import json
from gtts import gTTS

# Carpetas de entrada y salida
json_folder = "Bot_NLP/json_recording/"
audio_output_folder = "Bot_NLP/audio_confirmation/"

# Crear la carpeta si no existe
os.makedirs(audio_output_folder, exist_ok=True)

def generar_confirmacion_audio(json_file):
    file_path = os.path.join(json_folder, json_file)
    # Leer el contenido del JSON
    with open(file_path, "r", encoding="utf-8") as file:
        reserva = json.load(file)
    # Construir mensaje de confirmación
    mensaje = (
        f"Su reserva ha sido confirmada. {reserva['tickets']} boletos desde {reserva['source']} "
        f"hasta {reserva['destination']} el {reserva['date']} con la aerolinea {reserva['airline']}."
    )

    # Generar el archivo de audio
    tts = gTTS(text=mensaje, lang="es")
    audio_output_path = os.path.join(audio_output_folder, f"{os.path.splitext(json_file)[0]}_confirmacion.mp3")
    tts.save(audio_output_path)

    print(f"Confirmación de reserva generada en: {audio_output_path}")

# Procesar los archivos JSON
for filename in os.listdir(json_folder):
    if filename.endswith(".json"):
        generar_confirmacion_audio(filename)

✅ Confirmación de reserva generada en: Bot_NLP/audio_confirmation/audio_1b621ff126b341fdb8cedb63a53bdd68_converted_confirmacion.mp3


# Comentarios

## Recibe audios comentarios

In [15]:
import IPython
import uuid
from google.colab import output
from base64 import b64decode
import os

# Crear la carpeta si no existe
os.makedirs("./Bot_NLP/original_comment_recording/", exist_ok=True)

AUDIO_HTML = """
<script>
var my_audio = "";
var recorder, gumStream;

function startRecording() {
    navigator.mediaDevices.getUserMedia({ audio: true }).then(function(stream) {
        gumStream = stream;
        recorder = new MediaRecorder(stream);
        recorder.ondataavailable = function(event) {
            var reader = new FileReader();
            reader.readAsDataURL(event.data);
            reader.onloadend = function() {
                var base64data = reader.result.split(',')[1];
                google.colab.kernel.invokeFunction('notebook.save_audio', [base64data], {});
            }
        };
        recorder.start();
    });
}

function stopRecording() {
    recorder.stop();
    gumStream.getAudioTracks()[0].stop();
}
</script>

<button onclick="startRecording()"> Iniciar Grabación</button>
<button onclick="stopRecording()"> Detener Grabación</button>
"""

IPython.display.display(IPython.display.HTML(AUDIO_HTML))

# Función para guardar el audio en un archivo
def save_audio(base64data):
    audio_filename = f"./Bot_NLP/original_comment_recording/audio_{uuid.uuid4().hex}.wav"
    with open(audio_filename, "wb") as audio_file:
        audio_file.write(b64decode(base64data))
    print(f" Audio guardado en: {audio_filename}")
    return audio_filename

# Registrar la función en Google Colab
output.register_callback('notebook.save_audio', save_audio)

🎙 Audio guardado en: ./Bot_NLP/original_comment_recording/audio_d3ec5d70a55d48459c7455cb1e677e78.wav


In [2]:
import os
import glob
from pydub import AudioSegment

# Rutas de las carpetas
original_folder = "./Bot_NLP/original_comment_recording"
converted_folder = "./Bot_NLP/converted_comment_recording"

os.makedirs(converted_folder, exist_ok=True)

# Buscar archivo de audio
audio_files = glob.glob(f"{original_folder}/audio_*.wav")
print(f"Archivos de audio encontrados: {audio_files}")

# Función para convertir el audio a PCM WAV
def convert_to_wav_pcm(audio_path):
    filename = os.path.basename(audio_path).replace(".wav", "_converted.wav")
    output_path = os.path.join(converted_folder, filename)

    try:
        # Cargar el archivo de audio y convertirlo
        audio = AudioSegment.from_file(audio_path)
        audio = audio.set_channels(1).set_frame_rate(16000)
        audio.export(output_path, format="wav")
        print(f"Archivo convertido y guardado en: {output_path}")
        return output_path
    except Exception as e:
        print(f"Error al convertir el audio {audio_path}: {e}")
        return None

converted_audio_paths = [convert_to_wav_pcm(audio) for audio in audio_files]
converted_audio_paths = [path for path in converted_audio_paths if path is not None]

📂 Archivos de audio encontrados: ['./Bot_NLP/original_comment_recording/audio_d3ec5d70a55d48459c7455cb1e677e78.wav']
✅ Archivo convertido y guardado en: ./Bot_NLP/converted_comment_recording/audio_d3ec5d70a55d48459c7455cb1e677e78_converted.wav


## Realiza la transcripción de los audios de comentarios:

In [3]:
import whisper
import os

# Definir rutas de las carpetas
ruta_audios = "Bot_NLP/converted_comment_recording"
ruta_textos = "Bot_NLP/text_comment_recording"

os.makedirs(ruta_textos, exist_ok=True)

# Cargar modelo de Whisper
model = whisper.load_model("medium")

def transcribir_audio(audio_path, output_path):
    print(f"\n Transcribiendo: {audio_path} ...")
    result = model.transcribe(audio_path)
    texto_transcrito = result["text"]
    print("Transcripción:")
    print(texto_transcrito)

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(texto_transcrito)

    print(f"Transcripción guardada en: {output_path}")

for archivo in os.listdir(ruta_audios):
    if archivo.endswith((".wav")):
        audio_path = os.path.join(ruta_audios, archivo)
        texto_path = os.path.join(ruta_textos, os.path.splitext(archivo)[0] + ".txt")
        transcribir_audio(audio_path, texto_path)

print("\n Proceso de transcripción completado.")

100%|█████████████████████████████████████| 1.42G/1.42G [00:21<00:00, 71.1MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper


🎙️ Transcribiendo: Bot_NLP/converted_comment_recording/audio_d3ec5d70a55d48459c7455cb1e677e78_converted.wav ...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


📝 Transcripción:

✅ Transcripción guardada en: Bot_NLP/text_comment_recording/audio_d3ec5d70a55d48459c7455cb1e677e78_converted.txt

🎉 Proceso de transcripción completado.


## Clasifica los comentarios y genera JSON

In [8]:
import os
import json
from transformers import pipeline

# Configurar carpetas de entrada y salida
transcription_folder = "./Bot_NLP/text_comment_recording"
json_output_folder = "./Bot_NLP/json_comment_recording"

os.makedirs(json_output_folder, exist_ok=True)

# Cargar modelo de clasificación
classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

# Definir etiquetas de clasificación
labels = ["Queja", "Felicitación", "Otro"]

# Leer las transcripciones
for filename in os.listdir(transcription_folder):
    if filename.endswith(".txt"):
        file_path = os.path.join(transcription_folder, filename)
        # Leer el contenido del archivo de transcripción
        with open(file_path, "r", encoding="utf-8") as file:
            comentario = file.read().strip()
        # Clasificar el comentario
        resultado = classifier(comentario, labels)
        # Obtener la etiqueta más probable y la confianza
        categoria = resultado['labels'][0]
        confianza = resultado['scores'][0]
        data = {
            "comentario": comentario,
            "categoria": categoria,
            "confianza": confianza
        }

        # Guardar en archivo JSON
        json_filename = f"{os.path.splitext(filename)[0]}.json"
        json_path = os.path.join(json_output_folder, json_filename)

        with open(json_path, "w", encoding="utf-8") as json_file:
            json.dump(data, json_file, ensure_ascii=False, indent=4)

        print(f"Comentario clasificado y guardado en {json_filename}")

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu



🔹 Archivo: audio_d3ec5d70a55d48459c7455cb1e677e78_converted.txt
🔹 Comentario: El vuelo llegó tarde y la atención fue pésima.
✅ Clasificación: Queja (Confianza: 0.42)



## Almacena los comentarios y su clasificación en la base de datos

In [13]:
import os
import mysql.connector
import json

# Configurar los datos de conexión a SingleStore
conexion = mysql.connector.connect(
    host="svc-15fdff82-6c53-40e2-9dee-b597dfb154a5-dml.aws-oregon-4.svc.singlestore.com",
    user="admin",
    password="rdme6tYm4qYA35cZzaVYLZHDn6rRCVfP",
    database="Bot_NLP",
    port=3306
)

cursor = conexion.cursor()

# Carpeta donde están los JSON generados
json_folder = "./Bot_NLP/json_comment_recording"

# Función para insertar datos en la base de datos
def insertar_en_bd(comentario, categoria, confianza):
    try:
        sql = """
        INSERT INTO comentarios_clasificados (comentario, categoria, confianza)
        VALUES (%s, %s, %s)
        """
        cursor.execute(sql, (comentario, categoria, confianza))
        conexion.commit()
        print(f" Comentario almacenado en BD: {comentario[:50]}... ({categoria})")
    except Exception as e:
        print(f" Error al insertar en la base de datos: {e}")

# Procesar los archivos JSON en la carpeta
for filename in os.listdir(json_folder):
    if filename.endswith(".json"):
        file_path = os.path.join(json_folder, filename)
        # Leer el contenido del JSON
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        # Extraer datos
        comentario = data.get("comentario", "No especificado")
        categoria = data.get("categoria", "No clasificado")
        confianza = data.get("confianza", 0.0)
        # Almacenar en la base de datos
        insertar_en_bd(comentario, categoria, confianza)

# Cerrar conexión al finalizar
cursor.close()
conexion.close()
print("Proceso completado, conexión cerrada.")

✅ Comentario almacenado en BD: El vuelo llegó tarde y la atención fue pésima.... (Queja)
✅ Proceso completado, conexión cerrada.


# Telegram

In [8]:
!pip install python-telegram-bot

In [9]:
import nest_asyncio
nest_asyncio.apply()

from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, CallbackQueryHandler, filters, CallbackContext
import whisper
import json
import asyncio
import os
from gtts import gTTS
from telegram import InputFile

import mysql.connector
import json

# Configurar los datos de conexión
DB_CONFIG = {
    "host": "svc-15fdff82-6c53-40e2-9dee-b597dfb154a5-dml.aws-oregon-4.svc.singlestore.com",
    "user": "admin",
    "password": "rdme6tYm4qYA35cZzaVYLZHDn6rRCVfP",
    "database": "Bot_NLP",
    "port": 3306
}

# Ruta de la carpeta donde se guardan los archivos JSON
json_folder = "Bot_NLP/json_recording/"

audio_output_folder = "Bot_NLP/audio_confirmation/"
os.makedirs(audio_output_folder, exist_ok=True)

TOKEN = "7419389274:AAHOJjsNnqBvo3Jr4I78TDTl3FuL8S5sryQ"

modelo_whisper = whisper.load_model("medium")
reservas_pendientes = {}

async def procesar_mensaje(texto):
    from spacy.lang.es import Spanish
    nlp = Spanish()
    datos_reserva = extraer_informacion_a_json(texto)

    if isinstance(datos_reserva, str):
        try:
            datos_reserva = json.loads(datos_reserva)
        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON: {e}")
            datos_reserva = {}
    return datos_reserva


async def responder_texto(update: Update, context: CallbackContext) -> None:
    global reservas_pendientes

    mensaje_usuario = update.message.text
    datos_reserva = await procesar_mensaje(mensaje_usuario)

    # Convertir a diccionario si aún es un string JSON
    if isinstance(datos_reserva, str):
        try:
            datos_reserva = json.loads(datos_reserva)
        except json.JSONDecodeError:
            await update.message.reply_text("Error al procesar la reserva. Intenta nuevamente.")
            return

    # Obtener el user_id
    user_id = update.message.from_user.id

    # Guardar la reserva en `reservas_pendientes`
    reservas_pendientes[user_id] = datos_reserva

    print(f" Reserva guardada para user_id {user_id}: {reservas_pendientes}")

    # Crear botones de confirmación
    keyboard = [
        [InlineKeyboardButton("✅ Confirmar", callback_data="confirmar")],
        [InlineKeyboardButton("❌ Cancelar", callback_data="cancelar")]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    respuesta = (
        f"✈️ **Reserva detectada:**\n"
        f"🛫 Origen: {datos_reserva.get('source', 'No detectado')}\n"
        f"🛬 Destino: {datos_reserva.get('destination', 'No detectado')}\n"
        f"📅 Fecha: {datos_reserva.get('date', 'No detectado')}\n"
        f"🎫 Boletos: {datos_reserva.get('tickets', 1)}\n"
        f"🛩 Aerolínea: {datos_reserva.get('airline', 'No especificado')}\n\n"
        f"¿Deseas confirmar la reserva?"
    )

    await update.message.reply_text(respuesta, reply_markup=reply_markup, parse_mode="Markdown")

async def responder_audio(update: Update, context: CallbackContext) -> None:
    global reservas_pendientes

    archivo = await update.message.voice.get_file()
    await archivo.download_to_drive("mensaje.ogg")

    # Convertir OGG a WAV
    import ffmpeg
    ffmpeg.input("mensaje.ogg").output("mensaje.wav").run(overwrite_output=True)

    # Transcribir audio a texto
    resultado = modelo_whisper.transcribe("mensaje.wav")
    texto_transcrito = resultado["text"]

    # Procesar mensaje transcrito
    datos_reserva = await procesar_mensaje(texto_transcrito)

    # Verificar la estructura de datos_reserva antes de acceder a ella
    print(f" Tipo de datos_reserva después de procesar_mensaje: {type(datos_reserva)}")
    print(f" Contenido de datos_reserva: {datos_reserva}")

    # Forzar la conversión
    if isinstance(datos_reserva, str):
        try:
            datos_reserva = json.loads(datos_reserva)
        except json.JSONDecodeError as e:
            print(f"Error al convertir JSON: {e}")
            await update.message.reply_text("Error al procesar la reserva. Intenta nuevamente.")
            return

    # Verificar que datos_reserva sea un diccionario
    if not isinstance(datos_reserva, dict):
        await update.message.reply_text("Error al procesar la reserva. Intenta nuevamente.")
        return

    # Obtener el user_id correctamente
    user_id = update.message.from_user.id

    # Guardar la reserva en `reservas_pendientes`
    reservas_pendientes[user_id] = datos_reserva

    print(f"Reserva guardada para user_id {user_id}: {reservas_pendientes}")

    keyboard = [
        [InlineKeyboardButton("Confirmar", callback_data="confirmar")],
        [InlineKeyboardButton("Cancelar", callback_data="cancelar")]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    respuesta = (
        f"🎙 **Transcripción:** {texto_transcrito}\n\n"
        f"✈️ **Reserva detectada:**\n"
        f"🛫 Origen: {datos_reserva.get('source', 'No detectado')}\n"
        f"🛬 Destino: {datos_reserva.get('destination', 'No detectado')}\n"
        f"📅 Fecha: {datos_reserva.get('date', 'No detectado')}\n"
        f"🎫 Boletos: {datos_reserva.get('tickets', 1)}\n"
        f"🛩 Aerolínea: {datos_reserva.get('airline', 'No especificado')}\n\n"
        f"¿Deseas confirmar la reserva?"
    )

    await update.message.reply_text(respuesta, reply_markup=reply_markup, parse_mode="Markdown")

async def generar_confirmacion_audio(user_id, reserva, context):
    """Genera un archivo de audio con la confirmación de la reserva y lo envía al usuario en Telegram."""
    mensaje = (
        f"Su reserva ha sido confirmada. {reserva['tickets']} boletos desde {reserva['source']} "
        f"hasta {reserva['destination']} el {reserva['date']} con la aerolínea {reserva['airline']}."
    )

    # Generar archivo de audio
    audio_output_path = os.path.join(audio_output_folder, f"{user_id}_confirmacion.mp3")
    tts = gTTS(text=mensaje, lang="es")
    tts.save(audio_output_path)

    print(f"Confirmación de reserva generada en: {audio_output_path}")

    # Enviar audio al usuario en Telegram
    with open(audio_output_path, "rb") as audio_file:
        await context.bot.send_audio(chat_id=user_id, audio=InputFile(audio_file))

    # Eliminar el archivo después de enviarlo
    os.remove(audio_output_path)

# Función corregida para manejar la confirmación
async def manejar_confirmacion(update: Update, context: CallbackContext) -> None:
    global reservas_pendientes

    query = update.callback_query
    user_id = query.from_user.id

    print(f"Intentando recuperar reserva para user_id: {user_id}")
    print(f"Estado actual de reservas_pendientes: {reservas_pendientes}")

    # Verificar si hay una reserva pendiente
    reserva = reservas_pendientes.get(user_id)

    if reserva:
        # Extraer datos de la reserva
        source = reserva.get("source", "Sin Aeropuerto")
        destination = reserva.get("destination", "Sin Aeropuerto")
        tickets = reserva.get("tickets", 1)
        date = reserva.get("date", "")
        airline = reserva.get("airline", "No Especificado")

        # Insertar la reserva en la base de datos
        try:
            conexion = mysql.connector.connect(**DB_CONFIG)
            cursor = conexion.cursor()

            # Consulta SQL sin `user_id`
            sql = """
            INSERT INTO reservas_vuelos (source, destination, tickets, date, airline)
            VALUES (%s, %s, %s, %s, %s)
            """

            # Ahora pasamos solo los valores requeridos
            cursor.execute(sql, (source, destination, tickets, date, airline))
            print(f"✅ Insertado en BD: {source} a {destination}, {tickets} tickets con {airline}")

            conexion.commit()
            cursor.close()
            conexion.close()

            print(f"Reserva almacenada en la base de datos.")

        except mysql.connector.Error as e:
            print(f"Error al insertar en la base de datos: {e}")
            await query.edit_message_text("⚠️ Error al confirmar la reserva. Inténtalo nuevamente.")
            return

        # Confirmar la reserva con texto
        await query.edit_message_text("✅ ¡Reserva confirmada! ✈️\nGracias por tu compra.")

        # Generar y enviar confirmación en audio
        await generar_confirmacion_audio(user_id, reserva, context)

        # Eliminar la reserva del diccionario
        del reservas_pendientes[user_id]

    else:
        await query.edit_message_text("⚠️ No hay ninguna reserva pendiente.")

async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text("¡Hola! Soy tu asistente de reservas de vuelos. ✈️ Dime tu destino y fecha.")

def main():
    app = ApplicationBuilder().token(TOKEN).build()

    # Manejadores de comandos y mensajes
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, responder_texto))
    app.add_handler(MessageHandler(filters.VOICE, responder_audio))
    app.add_handler(CallbackQueryHandler(manejar_confirmacion))

    loop = asyncio.get_event_loop()
    loop.run_until_complete(app.run_polling())

if __name__ == '__main__':
    main()

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


📌 Tipo de datos_reserva después de procesar_mensaje: <class 'dict'>
📌 Contenido de datos_reserva: {'source': 'Quito', 'destination': 'Cuenca', 'tickets': 3, 'date': '25 de Enero de 2025', 'airline': 'No Especificado'}
✅ Reserva guardada para user_id 6240286376: {6240286376: {'source': 'Quito', 'destination': 'Cuenca', 'tickets': 3, 'date': '25 de Enero de 2025', 'airline': 'No Especificado'}}
📌 Intentando recuperar reserva para user_id: 6240286376
📌 Estado actual de reservas_pendientes: {6240286376: {'source': 'Quito', 'destination': 'Cuenca', 'tickets': 3, 'date': '25 de Enero de 2025', 'airline': 'No Especificado'}}
✅ Insertado en BD: Quito a Cuenca, 3 tickets con No Especificado
✅ Reserva almacenada en la base de datos.
✅ Confirmación de reserva generada en: Bot_NLP/audio_confirmation/6240286376_confirmacion.mp3


RuntimeError: Cannot close a running event loop